In [1]:
# 训练模型的3种方法
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import * 

2023-08-21 10:56:07.024290: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 10:56:07.059833: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 10:56:07.061097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 10:56:07.664922: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#打印时间分割线
@tf.function
def printbar():
    today_ts = tf.timestamp()%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)

    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))

    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8+timestring)

In [3]:
MAX_LEN = 300
BATCH_SIZE = 32
(x_train,y_train),(x_test,y_test) = datasets.reuters.load_data()
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen=MAX_LEN)
x_test = preprocessing.sequence.pad_sequences(x_test,maxlen=MAX_LEN)

MAX_WORDS = x_train.max()+1
CAT_NUM = y_train.max()+1

ds_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)) \
          .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

ds_test = tf.data.Dataset.from_tensor_slices((x_test,y_test)) \
          .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

2110848/2110848 [==============================] - 0s 0us/step


2023-08-21 10:56:40.051773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 10:56:40.076835: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# 内置fit方法
tf.keras.backend.clear_session()
def create_model():

    model = models.Sequential()
    model.add(layers.Embedding(MAX_WORDS,7,input_length=MAX_LEN))
    model.add(layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Conv1D(filters = 32,kernel_size = 3,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(CAT_NUM,activation = "softmax"))
    return(model)

def compile_model(model):
    model.compile(optimizer=optimizers.Nadam(),
                loss=losses.SparseCategoricalCrossentropy(),
                metrics=[metrics.SparseCategoricalAccuracy(),metrics.SparseTopKCategoricalAccuracy(5)]) 
    return(model)

model = create_model()
model.summary()
model = compile_model(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 7)            216874    
                                                                 
 conv1d (Conv1D)             (None, 296, 64)           2304      
                                                                 
 max_pooling1d (MaxPooling1  (None, 148, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 2336)              0

In [5]:
history = model.fit(ds_train,validation_data = ds_test,epochs = 10)

Epoch 1/10
281/281 [==============================] - 4s 9ms/step - loss: 2.0442 - sparse_categorical_accuracy: 0.4591 - sparse_top_k_categorical_accuracy: 0.7432 - val_loss: 1.6790 - val_sparse_categorical_accuracy: 0.5619 - val_sparse_top_k_categorical_accuracy: 0.7609
Epoch 2/10
281/281 [==============================] - 3s 9ms/step - loss: 1.4708 - sparse_categorical_accuracy: 0.6188 - sparse_top_k_categorical_accuracy: 0.7999 - val_loss: 1.5220 - val_sparse_categorical_accuracy: 0.6122 - val_sparse_top_k_categorical_accuracy: 0.7974
Epoch 3/10
281/281 [==============================] - 3s 9ms/step - loss: 1.1364 - sparse_categorical_accuracy: 0.7026 - sparse_top_k_categorical_accuracy: 0.8693 - val_loss: 1.5434 - val_sparse_categorical_accuracy: 0.6429 - val_sparse_top_k_categorical_accuracy: 0.8099
Epoch 4/10
281/281 [==============================] - 3s 9ms/step - loss: 0.8243 - sparse_categorical_accuracy: 0.7872 - sparse_top_k_categorical_accuracy: 0.9254 - val_loss: 1.7655 - 

In [6]:
# 内置train_on_batch方法
tf.keras.backend.clear_session()

def create_model():
    model = models.Sequential()

    model.add(layers.Embedding(MAX_WORDS,7,input_length=MAX_LEN))
    model.add(layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Conv1D(filters = 32,kernel_size = 3,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(CAT_NUM,activation = "softmax"))
    return(model)

def compile_model(model):
    model.compile(optimizer=optimizers.Nadam(),
                loss=losses.SparseCategoricalCrossentropy(),
                metrics=[metrics.SparseCategoricalAccuracy(),metrics.SparseTopKCategoricalAccuracy(5)]) 
    return(model)

model = create_model()
model.summary()
model = compile_model(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 7)            216874    
                                                                 
 conv1d (Conv1D)             (None, 296, 64)           2304      
                                                                 
 max_pooling1d (MaxPooling1  (None, 148, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 2336)              0

In [7]:
def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()

        # 在后期降低学习率
        if epoch == 5:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")

        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)

        if epoch%1 ==0:
            printbar()
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")

In [8]:
train_model(model,ds_train,ds_test,10)

================================================================================10:59:16
epoch =  1
train: {'loss': 1.3382827043533325, 'sparse_categorical_accuracy': 0.6363636255264282, 'sparse_top_k_categorical_accuracy': 0.8181818127632141}
valid: {'loss': 1.6594516038894653, 'sparse_categorical_accuracy': 0.5665785074234009, 'sparse_top_k_categorical_accuracy': 0.766313910484314}

================================================================================10:59:20
epoch =  2
train: {'loss': 0.8737804889678955, 'sparse_categorical_accuracy': 0.7727272510528564, 'sparse_top_k_categorical_accuracy': 0.9090909361839294}
valid: {'loss': 1.5068058967590332, 'sparse_categorical_accuracy': 0.6190476417541504, 'sparse_top_k_categorical_accuracy': 0.7989417910575867}

================================================================================10:59:24
epoch =  3
train: {'loss': 0.5218415856361389, 'sparse_categorical_accuracy': 0.8636363744735718, 'sparse_top_k_categorical_accuracy':

In [9]:
# 自定义训练循环
tf.keras.backend.clear_session()

def create_model():

    model = models.Sequential()

    model.add(layers.Embedding(MAX_WORDS,7,input_length=MAX_LEN))
    model.add(layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Conv1D(filters = 32,kernel_size = 3,activation = "relu"))
    model.add(layers.MaxPool1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(CAT_NUM,activation = "softmax"))
    return(model)

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 7)            216874    
                                                                 
 conv1d (Conv1D)             (None, 296, 64)           2304      
                                                                 
 max_pooling1d (MaxPooling1  (None, 148, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 146, 32)           6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 73, 32)            0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 2336)              0

In [10]:
optimizer = optimizers.Nadam()
loss_func = losses.SparseCategoricalCrossentropy()

train_loss = metrics.Mean(name='train_loss')
train_metric = metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = metrics.Mean(name='valid_loss')
valid_metric = metrics.SparseCategoricalAccuracy(name='valid_accuracy')

@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features,training = True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)


@tf.function
def valid_step(model, features, labels):
    predictions = model(features)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)


def train_model(model,ds_train,ds_valid,epochs):
    for epoch in tf.range(1,epochs+1):

        for features, labels in ds_train:
            train_step(model,features,labels)

        for features, labels in ds_valid:
            valid_step(model,features,labels)

        logs = 'Epoch={},Loss:{},Accuracy:{},Valid Loss:{},Valid Accuracy:{}'

        if epoch%1 ==0:
            printbar()
            tf.print(tf.strings.format(logs,
            (epoch,train_loss.result(),train_metric.result(),valid_loss.result(),valid_metric.result())))
            tf.print("")

        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

train_model(model,ds_train,ds_test,10)

================================================================================11:00:24
Epoch=1,Loss:2.00272822,Accuracy:0.474727243,Valid Loss:1.64700532,Valid Accuracy:0.579697251

================================================================================11:00:26
Epoch=2,Loss:1.4377737,Accuracy:0.632264555,Valid Loss:1.51856732,Valid Accuracy:0.621994674

================================================================================11:00:28
Epoch=3,Loss:1.130481,Accuracy:0.704186141,Valid Loss:1.57685363,Valid Accuracy:0.638468385

================================================================================11:00:31
Epoch=4,Loss:0.842117965,Accuracy:0.778111756,Valid Loss:1.86095572,Valid Accuracy:0.63089937

================================================================================11:00:33
Epoch=5,Loss:0.613242865,Accuracy:0.843576,Valid Loss:2.25635076,Valid Accuracy:0.621104181

================================================================================11:00

In [11]:
gpus = tf.config.list_physical_devices("GPU")
gpus

[]